In [1]:
from datasets import load_dataset
import torch


trainFile="Data/s288c_words_4.train.txt"

testText="112 211 121 121 112 112 122"

In [2]:
dataset=load_dataset('text', data_files=trainFile,split='train')

Using custom data configuration default-0ebeb9f8b6f204c9
Reusing dataset text (/home/lu/.cache/huggingface/datasets/text/default-0ebeb9f8b6f204c9/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)


In [3]:
batch_size = 1000
all_texts = [dataset[i : i + batch_size]["text"] for i in range(0, len(dataset), batch_size)]

In [4]:
def batch_iterator():
    for i in range(0, len(dataset), batch_size):
        yield dataset[i : i + batch_size]["text"]

In [5]:
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer

tokenizer = Tokenizer(models.WordPiece(unl_token="[UNK]"))

In [6]:
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)

In [7]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
)

In [20]:
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()

In [21]:
tokenizer.pre_tokenizer.pre_tokenize_str("331 131 331 111")

[('331', (0, 3)), ('131', (4, 7)), ('331', (8, 11)), ('111', (12, 15))]

In [22]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=2500, special_tokens=special_tokens)

In [23]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")

In [46]:
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

In [80]:
encoding = tokenizer.encode("3311 3133 1111")

In [81]:
encoding

Encoding(num_tokens=3, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [82]:
tokenizer.decoder = decoders.WordPiece(prefix="##")


In [83]:
encoding.tokens

['3311', '3133', '1111']

In [84]:
encoding.ids

[149, 315, 26]

'3311 3133 1111'

In [87]:
tokenizer.decoder = decoders.WordPiece(prefix="##")

In [ ]:
tokenizer.decode(encoding.ids)

In [89]:
from transformers import BertTokenizerFast

new_tokenizer = BertTokenizerFast(tokenizer_object=tokenizer)

In [90]:
new_tokenizer.save_pretrained("S288C_TOKENIZER_WORD")

('S288C_TOKENIZER_WORD/tokenizer_config.json',
 'S288C_TOKENIZER_WORD/special_tokens_map.json',
 'S288C_TOKENIZER_WORD/vocab.txt',
 'S288C_TOKENIZER_WORD/added_tokens.json',
 'S288C_TOKENIZER_WORD/tokenizer.json')

In [91]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /home/lu/.huggingface/token


In [92]:
new_tokenizer.push_to_hub("S288C_WordPiece_Tokenizer")

To https://huggingface.co/zluvolyote/S288C_WordPiece_Tokenizer
   18ce10a..b64ef62  main -> main



'https://huggingface.co/zluvolyote/S288C_WordPiece_Tokenizer/commit/b64ef62c7d58dc1b4157ef68e99b7fd630a0d74c'